## COVID-19 Dashboard

### Introduction
* This code to retrieves COVID-19 data (updated daily) from Johns Hopkins University's [GitHub repo](https://github.com/CSSEGISandData) and prepares it for use in Tableau
* You can find my COVID-19 dashboard here: [COVID-19 Dashboard](https://public.tableau.com/profile/alfred.zou#!/vizhome/COVID-19_15852011851070/COVID-19)
* Please feel free to download and play around with it. It runs faster in Tableau public than online

### Features
* Timeline of COVID-19 confirmed cases
* Comparison of new confirmed cases and deaths between countries and states
* The number of active cases vs. available ICU beds for some countries

### General Findings
#### Act Early
* At the start of a pandemic there are a lot of questions regarding if government actions are an overreaction
* However, there is 
* This usually leads to blame on governments for taking action too slowly

#### Number of Cases
* Its very difficult to determine the true number of cases
* 


#### Flattern the Curve
* There are two possible types of pandemics: a tall one and a short one. 
* There is also a horizontal line representing the total hospital capacity
* Flattening the curve means taking preventative measures to slow down the spread
* These could be government policies such as testing and lockdowns 
* Or hand washing, social distancing, remote work and reducing unnecessary trips
* Essentially squishing the tall curve below the hospital capacity line, relieving pressure off our medical system

<img src="Img/COVID-19.png" width="800" align="left"/>

* Approximately 5% of total COVID-19 cases are severe which require an ICU bed and a ventilator
* ICU 


<img src="Img/Severe.png" width="500" align="left"/>





### Good sources to learn more about COVID-19
* [Our World in Data](https://ourworldindata.org/coronavirus)
* [Coronavirus: Why You Must Act Now by Tomas Pueyo](https://medium.com/@tomaspueyo/coronavirus-act-today-or-people-will-die-f4d3d9cd99ca)
* [The Science Paper That Has Changed US and UK Covid-19 Policy by Noel Plum](https://www.youtube.com/watch?v=JbQ3oLvvecM)
* [How To Tell If We're Beating COVID-19 by Minute Physics](https://www.youtube.com/watch?v=54XLXg4fYsc)
* [The Coronavirus Explained & What You Should Do by Kurzgesagt](https://www.youtube.com/watch?v=BtN-goy9VOY)
* [Estimating actual COVID 19 cases (novel corona virus infections) in an area based on deaths by Khan Academy](https://www.youtube.com/watch?v=mCa0JXEwDEk)

### Data
* Data on coronavirus cases from Johns Hopkins University's githup repo: https://github.com/CSSEGISandData
* You can find their coronavirus dashboard here: https://coronavirus.jhu.edu/map.html
* Population estimates from: https://www.worldometers.info/world-population/population-by-country/
* US ICU beds estimate from AHA 2015 Annual Survey: https://www.sccm.org/Communications/Critical-Care-Statistics
* Mexico ICU beds estimate from: https://apnews.com/ff99a460e304ffc67ab03d8a57b2e1ef
* Australia and NZ ICU beds estimate from ANZICS 2018 report: https://www.anzics.com.au/wp-content/uploads/2019/10/2018-ANZICS-CORE-Report.pdf
* ICU beds estimate from 2012 academic paper 'The variability of critical care bed numbers in Europe' by Rhodes et al.: https://link.springer.com/article/10.1007/s00134-012-2627-8/tables/2
* ICU beds estimate from 2020 academic paper 'Critical Care Bed Capacity in Asian Countries and Regions' by Phua et al. :https://www.researchgate.net/figure/Number-of-Critical-Care-Beds_tbl1_338520008
* % Populations over 65 per country (2019): https://data.worldbank.org/indicator/SP.POP.65UP.TO.ZS
* Number of Doctors and Nurses per 10,000 people per country from 2015 WHO's World Health Stasticis report: https://apps.who.int/iris/bitstream/handle/10665/170250/9789240694439_eng.pdf;jsessionid=FC99DC52822E45FBA833E47587CB567E?sequence=1

### Code to extract and tranform data for Tableu

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Increase maximum rows displayed in pandas DataFrame
pd.set_option('display.max_rows', 9999)
pd.set_option('display.max_columns', 50)

In [3]:
# data from John Hopkin's University
# The data is a time series of confirmed cases and deaths
confirmed_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
                      "/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
confirmed_deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
             "/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
confirmed_recovered_url ="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data" \
            "/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [4]:
# Daily anity check if the erroneous data has been removed
foo = pd.read_csv(confirmed_cases_url)
foo[foo['Province/State']=='Recovered']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
238,Recovered,Canada,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Daily sanity check if the erroneous data has been removed
foo[foo['Province/State'].apply(str).str.contains('[Pp]rincess')]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
37,Grand Princess,Canada,37.6489,-122.6655,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,8,9,9,10,10,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
231,Diamond Princess,Canada,0.0000,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# Load data, remove wrong data and prepare data for importing into Tableau
confirmed_cases = pd.read_csv(confirmed_cases_url)
confirmed_cases.drop(confirmed_cases[confirmed_cases['Province/State']=='Recovered'].index,inplace=True)
confirmed_cases.drop(confirmed_cases[confirmed_cases['Province/State']=='Diamond Princess'].index,inplace=True)
confirmed_cases = confirmed_cases.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Cases')
confirmed_cases['Date']=pd.to_datetime(confirmed_cases['Date'],format='%m/%d/%y')
confirmed_cases.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_cases.reset_index(drop=True,inplace=True)
confirmed_cases

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
20353,NaN,Zimbabwe,-20.0,30.0,2020-04-04,9
20354,NaN,Zimbabwe,-20.0,30.0,2020-04-05,9
20355,NaN,Zimbabwe,-20.0,30.0,2020-04-06,10
20356,NaN,Zimbabwe,-20.0,30.0,2020-04-07,11


In [7]:
# Load data and remove wrong data
confirmed_deaths = pd.read_csv(confirmed_deaths_url)
confirmed_deaths.drop(confirmed_deaths[confirmed_deaths['Province/State']=='Recovered'].index,inplace=True)
confirmed_deaths.drop(confirmed_deaths[confirmed_deaths['Province/State']=='Diamond Princess'].index,inplace=True)
confirmed_deaths = confirmed_deaths.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Deaths')
confirmed_deaths['Date']=pd.to_datetime(confirmed_deaths['Date'],format='%m/%d/%y')
confirmed_deaths.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_deaths.reset_index(drop=True,inplace=True)
confirmed_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Deaths
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
20353,NaN,Zimbabwe,-20.0,30.0,2020-04-04,1
20354,NaN,Zimbabwe,-20.0,30.0,2020-04-05,1
20355,NaN,Zimbabwe,-20.0,30.0,2020-04-06,1
20356,NaN,Zimbabwe,-20.0,30.0,2020-04-07,2


In [8]:
# Load data and remove wrong data
confirmed_recovered = pd.read_csv(confirmed_recovered_url)
confirmed_recovered.drop(confirmed_recovered[confirmed_recovered['Province/State']=='Recovered'].index,inplace=True)
confirmed_recovered.drop(confirmed_recovered[confirmed_recovered['Province/State']=='Diamond Princess'].index,inplace=True)
confirmed_recovered = confirmed_recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed Recovered')
confirmed_recovered['Date']=pd.to_datetime(confirmed_recovered['Date'],format='%m/%d/%y')
confirmed_recovered.sort_values(['Country/Region','Province/State','Date'],inplace=True)
confirmed_recovered.reset_index(drop=True,inplace=True)
confirmed_recovered

,Province/State,Country/Region,Lat,Long,Date,Confirmed Recovered
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0
...,...,...,...,...,...,...
19417,NaN,Zimbabwe,-20.0,30.0,2020-04-04,0
19418,NaN,Zimbabwe,-20.0,30.0,2020-04-05,0
19419,NaN,Zimbabwe,-20.0,30.0,2020-04-06,0
19420,NaN,Zimbabwe,-20.0,30.0,2020-04-07,0


In [9]:
# Join the data together
confirmed_cases_deaths = confirmed_cases.merge(confirmed_deaths)
confirmed_cases_deaths = confirmed_cases_deaths.merge(confirmed_recovered)
confirmed_cases_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0,0,0
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0,0,0
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0,0,0
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0,0,0
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0,0,0
...,...,...,...,...,...,...,...,...
19105,NaN,Zimbabwe,-20.0,30.0,2020-04-04,9,1,0
19106,NaN,Zimbabwe,-20.0,30.0,2020-04-05,9,1,0
19107,NaN,Zimbabwe,-20.0,30.0,2020-04-06,10,1,0
19108,NaN,Zimbabwe,-20.0,30.0,2020-04-07,11,2,0


In [10]:
# Create arrays of unique country_region and province_state pairs to iterate through
countries_regions = confirmed_cases_deaths['Country/Region'].unique() 
provinces_states = confirmed_cases_deaths['Province/State'].unique()

# Create new columns
# Because countries will outbreak at different times, its useful to compare with the number of days since a country has reported X number of comulative cases
# Hence the new columns Days_Since_X. A day of 0 means setting the  
# New_Cases column is used to determine the daily number of new infections
confirmed_cases_deaths['Days Since 1 Cases'] = 0
confirmed_cases_deaths['Days Since 100 Cases'] = 0
confirmed_cases_deaths['Days Since 500 Cases'] = 0
confirmed_cases_deaths['Days Since 1000 Cases'] = 0
confirmed_cases_deaths['New Cases'] = 0
confirmed_cases_deaths['New Deaths'] = 0

# Iterating through each country/region, create a filtering mask
for cr in countries_regions:
    mask1 = confirmed_cases_deaths['Country/Region']== cr
    
    # Iterating through each province/state, create a filtering mask
    for ps in provinces_states:
        # This lets us filter for provinces/states that have a null value
        if type(ps)==float:
            mask2 = confirmed_cases_deaths['Province/State'].isnull()
        else:
            mask2 = confirmed_cases_deaths['Province/State']== ps
        
        # Filtering by applying country/region and province/state mask, find the difference between the Confirmed cases to get the new daily cases and deaths
        confirmed_cases_deaths.loc[mask1 & mask2,'New Cases'] =confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Cases'].diff()
        confirmed_cases_deaths.loc[mask1 & mask2,'New Deaths'] =confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Deaths'].diff()
        
        # Initialise empty count values
        # These will reset with each province/state
        count_1 = 0
        count_100 = 0     
        count_500 = 0
        count_1000 = 0   
        
        # For each row in the filtered df, check if the Confirmed cases have passed each threshold
        # If so, apply an accumalating count
        # If not, assign np.nan
        for row in confirmed_cases_deaths.loc[mask1 & mask2,'Confirmed Cases'].index:         
            if confirmed_cases_deaths.loc[row,'Confirmed Cases'] > 1000:
                confirmed_cases_deaths.loc[row,'Days Since 1000 Cases'] = count_1000
                count_1000 += 1
            else:
                confirmed_cases_deaths.loc[row,'Days Since 1000 Cases'] = np.nan

            if confirmed_cases_deaths.loc[row,'Confirmed Cases'] > 500:
                confirmed_cases_deaths.loc[row,'Days Since 500 Cases'] = count_500
                count_500 += 1
            else:
                confirmed_cases_deaths.loc[row,'Days Since 500 Cases'] = np.nan
                
            if confirmed_cases_deaths.loc[row,'Confirmed Cases'] > 100:
                confirmed_cases_deaths.loc[row,'Days Since 100 Cases'] = count_100
                count_100 += 1
            else:
                confirmed_cases_deaths.loc[row,'Days Since 100 Cases'] = np.nan
                               
            if confirmed_cases_deaths.loc[row,'Confirmed Cases'] > 1:
                confirmed_cases_deaths.loc[row,'Days Since 1 Cases'] = count_1
                count_1 += 1
            else:
                confirmed_cases_deaths.loc[row,'Days Since 1 Cases'] = np.nan
            
# Because the first row in new_cases is always going to be null using .diff(), assign new_cases = confirmed_cases_deaths for first row (or null) values
nan_mask = confirmed_cases_deaths['New Cases'].isnull()
confirmed_cases_deaths.loc[nan_mask,'New Cases'] = confirmed_cases_deaths['Confirmed Cases']
nan_mask = confirmed_cases_deaths['New Deaths'].isnull()
confirmed_cases_deaths.loc[nan_mask,'New Deaths'] = confirmed_cases_deaths['Confirmed Deaths']
# Convert floats into ints
confirmed_cases_deaths['New Cases']=pd.to_numeric(confirmed_cases_deaths['New Cases'],downcast='signed')

In [11]:
# Sanity check the Days Since X Cases
mask1 = confirmed_cases_deaths['Country/Region']== 'China'
mask2 = confirmed_cases_deaths['Province/State']== confirmed_cases_deaths.loc[mask1,'Province/State'].unique()[-1]
confirmed_cases_deaths.loc[mask1 & mask2]

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 1 Cases,Days Since 100 Cases,Days Since 500 Cases,Days Since 1000 Cases,New Cases,New Deaths
5772,Zhejiang,China,29.1832,120.0934,2020-01-22,10,0,0,0.0,NaN,NaN,NaN,10,0.0
5773,Zhejiang,China,29.1832,120.0934,2020-01-23,27,0,0,1.0,NaN,NaN,NaN,17,0.0
5774,Zhejiang,China,29.1832,120.0934,2020-01-24,43,0,1,2.0,NaN,NaN,NaN,16,0.0
5775,Zhejiang,China,29.1832,120.0934,2020-01-25,62,0,1,3.0,NaN,NaN,NaN,19,0.0
5776,Zhejiang,China,29.1832,120.0934,2020-01-26,104,0,1,4.0,0.0,NaN,NaN,42,0.0
5777,Zhejiang,China,29.1832,120.0934,2020-01-27,128,0,1,5.0,1.0,NaN,NaN,24,0.0
5778,Zhejiang,China,29.1832,120.0934,2020-01-28,173,0,3,6.0,2.0,NaN,NaN,45,0.0
5779,Zhejiang,China,29.1832,120.0934,2020-01-29,296,0,3,7.0,3.0,NaN,NaN,123,0.0
5780,Zhejiang,China,29.1832,120.0934,2020-01-30,428,0,4,8.0,4.0,NaN,NaN,132,0.0
5781,Zhejiang,China,29.1832,120.0934,2020-01-31,538,0,14,9.0,5.0,0.0,NaN,110,0.0


In [12]:
# Sanity check the Days Since X Cases
mask1 = confirmed_cases_deaths['Country/Region']== 'India'
confirmed_cases_deaths.loc[mask1]

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 1 Cases,Days Since 100 Cases,Days Since 500 Cases,Days Since 1000 Cases,New Cases,New Deaths
9984,NaN,India,21.0,78.0,2020-01-22,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9985,NaN,India,21.0,78.0,2020-01-23,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9986,NaN,India,21.0,78.0,2020-01-24,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9987,NaN,India,21.0,78.0,2020-01-25,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9988,NaN,India,21.0,78.0,2020-01-26,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9989,NaN,India,21.0,78.0,2020-01-27,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9990,NaN,India,21.0,78.0,2020-01-28,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9991,NaN,India,21.0,78.0,2020-01-29,0,0,0,NaN,NaN,NaN,NaN,0,0.0
9992,NaN,India,21.0,78.0,2020-01-30,1,0,0,NaN,NaN,NaN,NaN,1,0.0
9993,NaN,India,21.0,78.0,2020-01-31,1,0,0,NaN,NaN,NaN,NaN,0,0.0


In [13]:
# Load in data which maps countries to continents
continent_countries = pd.read_excel('Countries.xlsx',usecols = ['Continent','Country','ICU Beds','Doctors per 10000','Nurses per 10000','ICU Beds per 100000'])

In [14]:
# Map the continents to the countries
confirmed_cases_deaths = confirmed_cases_deaths.merge(continent_countries,how='left',left_on='Country/Region',right_on='Country')
confirmed_cases_deaths.drop(columns='Country',inplace=True)
confirmed_cases_deaths

,Province/State,Country/Region,Lat,Long,Date,Confirmed Cases,Confirmed Deaths,Confirmed Recovered,Days Since 1 Cases,Days Since 100 Cases,Days Since 500 Cases,Days Since 1000 Cases,New Cases,New Deaths,Continent,ICU Beds,Doctors per 10000,Nurses per 10000,ICU Beds per 100000
0,NaN,Afghanistan,33.0,65.0,2020-01-22,0,0,0,NaN,NaN,NaN,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
1,NaN,Afghanistan,33.0,65.0,2020-01-23,0,0,0,NaN,NaN,NaN,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
2,NaN,Afghanistan,33.0,65.0,2020-01-24,0,0,0,NaN,NaN,NaN,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
3,NaN,Afghanistan,33.0,65.0,2020-01-25,0,0,0,NaN,NaN,NaN,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
4,NaN,Afghanistan,33.0,65.0,2020-01-26,0,0,0,NaN,NaN,NaN,NaN,0,0.0,Asia,NaN,2.7,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19105,NaN,Zimbabwe,-20.0,30.0,2020-04-04,9,1,0,14.0,NaN,NaN,NaN,0,0.0,Africa,NaN,0.8,13.4,NaN
19106,NaN,Zimbabwe,-20.0,30.0,2020-04-05,9,1,0,15.0,NaN,NaN,NaN,0,0.0,Africa,NaN,0.8,13.4,NaN
19107,NaN,Zimbabwe,-20.0,30.0,2020-04-06,10,1,0,16.0,NaN,NaN,NaN,1,0.0,Africa,NaN,0.8,13.4,NaN
19108,NaN,Zimbabwe,-20.0,30.0,2020-04-07,11,2,0,17.0,NaN,NaN,NaN,1,1.0,Africa,NaN,0.8,13.4,NaN


In [15]:
pop_above_65 = pd.read_csv('Pop Above 65.csv')
pop_above_65.columns = ['Country/Region','% Population Above 65']
confirmed_cases_deaths = confirmed_cases_deaths.merge(pop_above_65,how='left')

In [16]:
pop_above_65.sort_values('% Population Above 65',ascending=False)

,Country/Region,% Population Above 65
117,Japan,27.576370
114,Italy,22.751680
192,Portugal,21.953858
73,Finland,21.720788
87,Greece,21.655272
53,Germany,21.461962
19,Bulgaria,21.021914
66,Euro area,20.600288
97,Croatia,20.445433
157,Malta,20.349324


In [17]:
def create_ref_line(starting_value):
    '''
    Creates two reference points for Tableau to draw its trendline for a certain starting_value of confirmed cases
    '''
    length = 80
    x = list(range(length))
    y = np.zeros(length)
    df = pd.DataFrame({f'Days Since {starting_value} Cases':x,'Doubles in 2 Days':y,'Doubles in 3 Days':y,'Doubles in 4 Days':y,'Doubles in 5 Days':y,
                        'Doubles in 6 Days':y,'Doubles in 7 Days':y,'Doubles in 8 Days':y,'Doubles in 9 Days':y,'Doubles in 10 Days':y})
    df.iloc[0,1:] = starting_value
    
    for row in range(1,length):
        for step in range(2,11):
            df.loc[row,f'Doubles in {step} Days'] = df.loc[row-1,f'Doubles in {step} Days']*pow(2,1/step)
    return df

In [18]:
def ref_line_df(list_of_starting_values):
    '''
    Creates and concatenates all reference points for a list of starting values
    '''
    count = 0
    for i in list_of_starting_values:
        if count == 0:
            df = create_ref_line(i)
            count += 1
        else:
            df = pd.concat([df,create_ref_line(i)],sort=False)
    return df

In [19]:
reference_lines = ref_line_df([1,100,500,1000])
reference_lines = reference_lines.melt(id_vars=['Days Since 1000 Cases','Days Since 500 Cases','Days Since 100 Cases','Days Since 1 Cases']
                                       ,var_name='Doubles in N Days',value_name='Reference Cases')
reference_lines

,Days Since 1000 Cases,Days Since 500 Cases,Days Since 100 Cases,Days Since 1 Cases,Doubles in N Days,Reference Cases
0,NaN,NaN,NaN,0.0,Doubles in 2 Days,1.000000e+00
1,NaN,NaN,NaN,1.0,Doubles in 2 Days,1.414214e+00
2,NaN,NaN,NaN,2.0,Doubles in 2 Days,2.000000e+00
3,NaN,NaN,NaN,3.0,Doubles in 2 Days,2.828427e+00
4,NaN,NaN,NaN,4.0,Doubles in 2 Days,4.000000e+00
5,NaN,NaN,NaN,5.0,Doubles in 2 Days,5.656854e+00
6,NaN,NaN,NaN,6.0,Doubles in 2 Days,8.000000e+00
7,NaN,NaN,NaN,7.0,Doubles in 2 Days,1.131371e+01
8,NaN,NaN,NaN,8.0,Doubles in 2 Days,1.600000e+01
9,NaN,NaN,NaN,9.0,Doubles in 2 Days,2.262742e+01


In [20]:
# Create folder and save csv's for Tableau to read
!mkdir Data
confirmed_cases_deaths.to_csv('Data/confirmed_cases_deaths.csv')
reference_lines.to_csv('Data/reference_lines.csv')

A subdirectory or file Data already exists.
